In [1]:
base_path="../eit_basic_experiments/exp_0.1" #Don't add '/' at start and end
T=200 #Training time period
C=1000000 #Capital available
file=1 #Index_file
m=8 #Inital Size of Kernel
lbuck=2 #Size Of Buckets
pho=0.1 #Transaction Cost Proportion
lamda=1/(1000*C) # lower_bound for capital invested in jth stock
nuh=0.8  # upper_bound
xii=1.2 # Limit for Tracking Error
k=14 #Cardinality constraint for stocks in portfolio
p=3 #If stock not selected in optimal soln in last p iters then it is to be dropped from kernel
f=12
#cj,cb are declared in script

In [2]:
import sys
sys.path.insert(0,"..") #Add root folder to sys path

from eit_basic_experiment import TestEitBasic
import time
from tqdm.notebook import tqdm
from IPython.utils import io
from joblib import Parallel, delayed
import pandas as pd

In [3]:
test_eit_basic=TestEitBasic(output=base_path,T=T,C=C,file=file,m=m,lbuck=lbuck,\
                                        pho=pho,lamda=lamda,nuh=nuh,xii=xii,k=k,p=p,f=f)

In [4]:
test_eit_basic.give_params()

params(output='../eit_basic_experiments/exp_0.1', T=200, C=1000000, file=1, m=8, lbuck=2, pho=0.1, lamda=1e-09, nuh=0.8, xii=1.2, k=14, p=3, f=12)

In [5]:
%%time
result=test_eit_basic.step_1(from_root=False,verbose=True)

+----------------------------------------------------+
    Step 1: Solving Linear Relaxation of EIT-Basic
+----------------------------------------------------+
+----------------------------------------------------+
    Step 1 complete in 1.99s
+----------------------------------------------------+
CPU times: user 5.21 ms, sys: 2.41 ms, total: 7.62 ms
Wall time: 1.99 s


In [6]:
result[1]

1653.519570394083

In [7]:
result_2a=test_eit_basic.step_2a(result[0],result[1],result[2],from_root=False,verbose=True)

+----------------------------------------------------+
    Step 2a: Sort Securities and create buckets
+----------------------------------------------------+


/Users/ashish1610dhiman/opt/anaconda3/envs/eit_paper/lib/python3.8/site-packages/pulp/pulp.py:1198: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")
../src/sort_and_buckets.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  not_optimal.sort_values(by="value",ascending=False,inplace=True)


In [8]:
result_2b=test_eit_basic.step_2b(kernel=result_2a[0], buckets=result_2a[1], from_root=False, verbose=True)

+----------------------------------------------------+
     Step 2b: Solve EIT(kernel) and get lower-bound
+----------------------------------------------------+
Solving EIT(kernel)
***************************************************
***************************************************

Optimisation Status=0,Objective Value=1300.8276541096857
OPTIMAL(0), ERROR(-1), INFEASIBLE(1), UNBOUNDED(2)


In [9]:
result_3=test_eit_basic.step_3(kernel=result_2a[0], L=result_2a[2],z = result_2b[0],\
                              Nb=result_2a[3], buckets=result_2a[1],\
                              failure = result_2b[1],execution_result = result_2b[-1],from_root=False, verbose=True)

+----------------------------------------------------+
    Step 3: Execution Phase of Kernel Search
+----------------------------------------------------+


For bucket=1
Solving EIT(kernel+bucket-1)
***************************************************
Optimisation Status=OptimizationStatus.OPTIMAL,Objective Value=1479.0305487388523
OPTIMAL(0), ERROR(-1), INFEASIBLE(1), UNBOUNDED(2)
Following stocks from bucket added to kernel:
security_15
security_29
***************************************************
Updating Lower Bound
Updating Kernel
Length of Old Kernel=8
Length of Updated Kernel=10
Current Length Kernel=10


For bucket=2
Solving EIT(kernel+bucket-2)
***************************************************
Optimisation Status=OptimizationStatus.OPTIMAL,Objective Value=1589.7274557883093
OPTIMAL(0), ERROR(-1), INFEASIBLE(1), UNBOUNDED(2)
Following stocks from bucket added to kernel:
security_23
security_8
***************************************************
Updating Lower Bound
Updating K

In [10]:
%%time
%%capture
result_df=test_eit_basic.run_experiment(from_root=False,verbose=False);
result_df["z_linear"]=test_eit_basic.objective_linear

CPU times: user 17.4 s, sys: 400 ms, total: 17.8 s
Wall time: 17.9 s


In [11]:
result_df

,bucket,kernel_size,problem_status,z_value,in_excess_return,in_tr_err,out_excess_return,out_tr_err,portfolio_size,z_linear
0,0,8,OptimizationStatus.OPTIMAL,1300.827654,1300.827654,6070.593586,-1200.760969,77870.084195,8,1653.51957
1,1,10,OptimizationStatus.OPTIMAL,1479.030549,1479.030549,6132.996429,-672.281984,29282.395125,10,1653.51957
2,2,12,OptimizationStatus.OPTIMAL,1589.727456,1589.727456,6101.203852,-457.331436,68895.694274,12,1653.51957
3,3,14,OptimizationStatus.OPTIMAL,1628.268633,1628.268633,6102.023306,-679.423036,47861.344100,14,1653.51957
4,4,15,OptimizationStatus.OPTIMAL,1633.800079,1633.800079,6097.179545,-619.789843,59883.101926,14,1653.51957
5,5,15,OptimizationStatus.INFEASIBLE,NaN,NaN,NaN,NaN,NaN,None,1653.51957
6,6,15,OptimizationStatus.INFEASIBLE,NaN,NaN,NaN,NaN,NaN,None,1653.51957
7,7,15,OptimizationStatus.INFEASIBLE,NaN,NaN,NaN,NaN,NaN,None,1653.51957
8,8,15,OptimizationStatus.INFEASIBLE,NaN,NaN,NaN,NaN,NaN,None,1653.51957
9,9,15,OptimizationStatus.INFEASIBLE,NaN,NaN,NaN,NaN,NaN,None,1653.51957


In [12]:
1300.827654/6070.593586

0.21428343630184174

In [13]:
1633.800079/6097.179545

0.267959975090417